<a href="https://colab.research.google.com/github/SY-256/basics-of-image-recognition/blob/main/chapter02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import torch
import torchvision.transforms as transforms

from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

In [ ]:
### 全結合層
class FullyConnectedLayer:
    def __init__(self, input_dim, output_dim):
        self.weights = np.random.randn(input_dim, output_dim) # 標準正規分布に従う乱数で重み作成
        self.bias = np.zeros(output_dim) # バイアスは0で初期化

    def forward(self, X):
        self.X = X
        return np.dot(X, self.weights) + self.bias # X * W.T + bias

    def backward(self, grad_output):
        """grad_output: ネットワーク上層から伝播してきた誤差勾配情報"""
        grad_input = np.dot(grad_output, self.weights.T)
        self.grad_weights = np.dot(self.X.T, grad_output)
        # バイアスの逆伝播ではミニバッチサイズ分の勾配を合算して求める
        self.grad_bias = np.sum(grad_output, axis=0)
        return grad_input

    def update(self, learning_rate):
        self.weights -= learning_rate * self.grad_weights
        self.bias -= learning_rate * self.grad_bias


In [ ]:
### ReLUの実装
class ReLU:
    def __init__(self):
        pass

    def forward(self, input_data):
        self.input = input_data
        return np.maximum(0, input_data)

    def backward(self, d_output):
        grad_input = d_output.copy()
        grad_input[self.input < 0] = 0
        return grad_input

In [ ]:
### 損失関数の実装（Softmax, CrossEntorpy）
class Softmax:
    def __init__(self):
        pass

    def forward(self, input_data, axis=-1):
        exps = np.exp(input_data - np.max(input_data, axis=axis, keepdims=True))
        output = exps / np.sum(exps, axis=axis, keepdims=True)
        return output

class CrossEntorpyLoss_With_Softmax:
    def __init__(self):
        self.sotfmax = Softmax()

    def forward(self, x, y):
        self.y = y
        self.y_hat = self.sotfmax.forward(x)
        return -np.sum(np.log(self.y_hat[np.argmax(self.y_hat.shape[0]), y] + 1e-8)) / self.y_hat.shape[0]


    def backward(self):
        batch_size = self.y.shape[0]
        dx = self.y_hat.copy()
        dx[np.arange(batch_size), self.y] -= 1
        dx = dx / batch_size
        return dx

In [ ]:
### 2層MLP

class MLP:
    def __init__(self, input_dim, hidden_dim, output_dim):
        self.layers = [
            FullyConnectedLayer(input_dim, hidden_dim),
            ReLU(),
            FullyConnectedLayer(hidden_dim, output_dim),
        ]

    def forward(self, X):
        for layer in self.layers:
            X = layer.forward(X)
        return X

    def backward(self, grad_output):
        for layer in reversed(self.layers):
            grad_output = layer.backward(grad_output)
        return grad_output

    def update(self, learning_rate):
        for layer in self.layers:
            if isinstance(layer, FullyConnectedLayer):
                layer.update(learning_rate)


In [ ]:
### 画像分類の実装

mnist_data = MNIST('./mnist', train=True, download=True, transform=transforms.ToTensor())
data_loader = DataLoader(mnist_data, batch_size=256, shuffle=True)


num_epoch = 50
learning_rate = 0.05

# MLPクラスの生成
mlp = MLP(784, 128, 10)

criterion = CrossEntorpyLoss_With_Softmax()

for epoch in range(num_epoch):
    loss = 0.0
    accuracy = 0.0
    for i, (X_mini, y_mini) in enumerate(data_loader):
        X_mini = X_mini.view(X_mini.size(0), -1).numpy()
        y_mini = y_mini.numpy()
        y_pred = mlp.forward(X_mini)
        loss += criterion.forward(y_pred, y_mini)

        # backward
        grad_output = criterion.backward()
        mlp.backward(grad_output)

        # update the weight parameters
        mlp.update(learning_rate)

        # Caluculate Accuracy
        y_pred = np.argmax(y_pred, axis=1)
        accuracy += np.mean(y_mini == y_pred) * 100

    loss /= len(data_loader)
    accuracy /= len(data_loader)

    print(f"Epoch [{epoch+1}/{num_epoch}], Loss: {loss}, Accuracy: {accuracy}%")


Epoch [1/50], Loss: 16.020813101624107, Accuracy: 70.00387854609929%
Epoch [2/50], Loss: 16.26834431750287, Accuracy: 83.5017730496454%
Epoch [3/50], Loss: 16.05271332989258, Accuracy: 85.88929521276596%
Epoch [4/50], Loss: 16.041624964942205, Accuracy: 87.14649822695036%
Epoch [5/50], Loss: 16.10248491769695, Accuracy: 88.05629432624114%
Epoch [6/50], Loss: 15.891021856012214, Accuracy: 88.62311613475178%
Epoch [7/50], Loss: 15.98930700355997, Accuracy: 89.04310726950354%
Epoch [8/50], Loss: 15.894130099237593, Accuracy: 89.51906028368795%
Epoch [9/50], Loss: 15.745977301502867, Accuracy: 89.74346187943263%
Epoch [10/50], Loss: 15.611353207804166, Accuracy: 90.00221631205673%
Epoch [11/50], Loss: 15.640237730577871, Accuracy: 90.1845079787234%
Epoch [12/50], Loss: 15.62040218437112, Accuracy: 90.38674645390071%
Epoch [13/50], Loss: 15.326432971026975, Accuracy: 90.48592641843972%
Epoch [14/50], Loss: 15.547506219375451, Accuracy: 90.71919326241135%
Epoch [15/50], Loss: 15.433497542646